In [1]:
import requests
from requests.auth import HTTPBasicAuth
from atl_module.secret_vars import COPERNICUS_PW, COPERNICUS_USERNAME
import xarray as xr
import geopandas as gpd
import pandas as pd
from atl_module import ocean_color

In [2]:
# dapurl = 'https://nrt.cmems-du.eu/thredds/dodsC/cmems_obs-oc_glo_bgc-transp_nrt_l4-gapfree-multi-4km_P1D'
dapurl = "https://my.cmems-du.eu/thredds/dodsC/cmems_obs-oc_glo_bgc-transp_my_l4-gapfree-multi-4km_P1D"
session = requests.session()
session.auth = (COPERNICUS_USERNAME, COPERNICUS_PW)

store = xr.backends.PydapDataStore.open(dapurl, session=session)

ds = xr.open_dataset(store)

subset = ds.sel(lat=24.69478, lon=-81.10013, method="nearest", tolerance=0.05).sel(
    time="2022-06-01", method="nearest"
)

In [8]:
ds.KD490_uncertainty

<xarray.Dataset>
Dimensions:            (lat: 4320, lon: 8640, time: 9030)
Coordinates:
  * lat                (lat) float32 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon                (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time               (time) datetime64[ns] 1997-09-04 ... 2022-06-17
Data variables:
    KD490              (time, lat, lon) float32 ...
    KD490_uncertainty  (time, lat, lon) float32 ...
    flags              (time, lat, lon) int8 ...
    ZSD                (time, lat, lon) float32 ...
    ZSD_uncertainty    (time, lat, lon) float32 ...
Attributes: (12/91)
    lon_step:                        0.041666668
    keywords:                        EARTH SCIENCE > OCEANS > OCEAN OPTICS > ...
    cmems_product_id:                OCEANCOLOUR_GLO_BGC_L4_MY_009_104
    lat_step:                        0.041666668
    grid_mapping:                    Equirectangular
    title:                           cmems_obs-oc_glo_bgc-transp_my_l4-gapfre...
    ...                              ...
    contact:                         servicedesk.cmems@acri-st.fr
    westernmost_longitude:           -180.0
    geospatial_vertical_positive:    up
    date_created:                    2022-07-12T08:51:00Z
    nb_valid_bins:                   18708515
    pct_valid_bins:                  50.123550561556925

In [2]:
zsd, zsd_sigma = ocean_color.get_zsd_info(
    24.69478, -81.10013, ["2022-06-01", "2022-05-01"]
)
zsd

array([10.204417, 10.187806], dtype=float32)

In [2]:
ocean_color.get_color_dataframe(24.69478, -81.10013, ["2022-06-01", "2022-05-01"])

,lat,lon,KD490,KD490_uncertainty,flags,ZSD,ZSD_uncertainty
time,,,,,,,
2022-06-01,24.687498,-81.104164,0.080234,79.379997,0,10.204417,61.379997
2022-05-01,24.687498,-81.104164,0.080346,80.860001,0,10.187806,62.859997


In [90]:
temp = tracklines.assign(
    xcoord=tracklines.to_crs("EPSG:4326").centroid.x,
    ycoord=tracklines.to_crs("EPSG:4326").centroid.y,
    date=pd.to_datetime(tracklines.date).dt.date,
)
results_per_track = temp.loc[:, ["xcoord", "ycoord", "date"]].apply(
    lambda x: pd.Series(
        ocean_color.get_zsd_info(x.ycoord, x.xcoord, x.date),
        index=["ZSD", "ZSD_uncertainty"],
    ),
    axis=1,
    result_type="expand",
)
tracklines = tracklines.assign(
    secchi_depth=results_per_track.ZSD,
    secchi_depth_unc=results_per_track.ZSD_uncertainty,
)

In [14]:
tracklines = gpd.read_file("../data/test_sites/maldives/tracklines.gpkg")
newdf = ocean_color.add_secchi_depth_to_tracklines(tracklines)

/mnt/c/Users/XCB/OneDrive - Van Oord/Documents/thesis/code/atl_module/ocean_color.py:59: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xcoord=df_input.to_crs("EPSG:4326").centroid.x,
/mnt/c/Users/XCB/OneDrive - Van Oord/Documents/thesis/code/atl_module/ocean_color.py:60: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ycoord=df_input.to_crs("EPSG:4326").centroid.y,


In [16]:
newdf.sort_values("secchi_depth")

,file,beam,Reference Ground Track,date,beam_type,geometry,secchi_depth,secchi_depth_unc
148,processed_ATL03_20211206034816_11411301_005_01,gt2r,1141,2021-12-06T03:48:16.410275Z,weak,"LINESTRING (73.46004 3.82581, 73.42956 4.12964)",5.785692,37.39
147,processed_ATL03_20211206034816_11411301_005_01,gt2l,1141,2021-12-06T03:48:16.410275Z,strong,"LINESTRING (73.45920 3.82591, 73.42871 4.12974)",5.785692,37.39
145,processed_ATL03_20211206034816_11411301_005_01,gt1l,1141,2021-12-06T03:48:16.410275Z,strong,"LINESTRING (73.43034 3.82591, 73.39985 4.12976)",6.346222,50.83
146,processed_ATL03_20211206034816_11411301_005_01,gt1r,1141,2021-12-06T03:48:16.410275Z,weak,"LINESTRING (73.43111 3.82599, 73.40064 4.12964)",6.346222,50.83
149,processed_ATL03_20211206034816_11411301_005_01,gt3l,1141,2021-12-06T03:48:16.410275Z,strong,"LINESTRING (73.48810 3.82591, 73.45760 4.12975)",8.5546055,55.74
...,...,...,...,...,...,...,...,...
20,processed_ATL03_20190313033014_11410201_005_01,gt3l,1141,2019-03-13T03:30:13.910601Z,strong,"LINESTRING (73.47570 3.82594, 73.44528 4.12978)",24.851147,50.29
112,processed_ATL03_20210308164832_11411001_005_01,gt1r,1141,2021-03-08T16:48:32.230888Z,strong,"LINESTRING (73.43120 3.82600, 73.40075 4.12966)",24.987423,10.13
114,processed_ATL03_20210308164832_11411001_005_01,gt2r,1141,2021-03-08T16:48:32.230888Z,strong,"LINESTRING (73.46012 3.82603, 73.42977 4.12869)",25.679447,18.65
116,processed_ATL03_20210308164832_11411001_005_01,gt3r,1141,2021-03-08T16:48:32.230888Z,strong,"LINESTRING (73.48910 3.82584, 73.45864 4.12967)",26.831112,25.27


In [ ]:
ds1 = ocean_color.ds.sel(lat=51.944184, lon=4.026319, method="nearest").sel(
    time="2022-01-30"
)